In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sn
import folium
import math
import json
from branca.colormap import LinearColormap

In [2]:
intercountry_df = pd.read_csv('../Data_sinan/intercountry_tone_count_table', dtype={'EventRootCode': np.str})
world_borders = json.load(open('../Data/world_borders.topojson.json'))

In [3]:
intercountry_df = intercountry_df[pd.notnull(intercountry_df['LocEvent'])]
intercountry_df = (intercountry_df[intercountry_df["EventRootCode"]!='--'])

In [4]:
world_map = folium.Map(tiles='Mapbox bright', zoom_start=5.5)

# Defining a style function to color the map and the borders
style_func=lambda feature: {'fillColor': '#990033','color' : 'black','weight' : 2 }

# Using a GeoJsonTooltip to display cantons' name when mouse is over them
tool=folium.GeoJsonTooltip(fields=['name'], aliases=['Country:'], sticky=True, \
                           style="font-family: Arial; color: black;", \
                           opacity=0.8, \
                           direction='top' \
                          )

# Load border information on the map
folium.TopoJson(world_borders,\
                object_path='objects.countries1',\
                style_function=style_func,\
                tooltip=tool, \
                overlay=False \
               ) \
      .add_to(world_map)

In [5]:
fips_codes = pd.read_csv('../Data/fipscountries.csv')
fips_codes = fips_codes[['fips','country']]

In [6]:
intercountry_df = fips_codes.merge(intercountry_df, left_on='fips', right_on='LocEvent', how='inner')
intercountry_df.loc[intercountry_df['country']=='United States', 'country'] = 'United States of America'
intercountry_df.loc[intercountry_df['country']=='Serbia', 'country'] = 'Republic of Serbia'

In [7]:
intercountry_nb_mentions_df = intercountry_df[['country', 'NumberOfInterMentions']]
intercountry_nb_mentions_df = intercountry_nb_mentions_df.groupby('country').sum()

In [8]:
intercountry_nb_mentions_df = intercountry_df[['country', 'NumberOfInterMentions']]
intercountry_nb_mentions_df = intercountry_nb_mentions_df.groupby('country', as_index=False).sum()
#intercountry_nb_mentions_df['NumberOfInterMentions'] = (intercountry_nb_mentions_df['NumberOfInterMentions']-min(intercountry_nb_mentions_df['NumberOfInterMentions'])) \
                                                       #/(max(intercountry_nb_mentions_df['NumberOfInterMentions'])-min(intercountry_nb_mentions_df['NumberOfInterMentions']))                                                                                    

intercountry_AVG_tone_df = intercountry_df[['country', 'AverageTone']]
intercountry_AVG_tone_df = intercountry_AVG_tone_df.groupby('country', as_index=False).mean()

In [9]:
intercountry_nb_mentions_df_dict = intercountry_nb_mentions_df.set_index('country')['NumberOfInterMentions'].to_dict()
intercountry_AVG_tone_df_dict = intercountry_AVG_tone_df.set_index('country')['AverageTone'].to_dict()


In [10]:
color_scale = LinearColormap(['yellow','red'], vmin = min(intercountry_nb_mentions_df_dict.values()), \
                                               vmax = max(intercountry_nb_mentions_df_dict.values()) \
                            )

def color_function(feature):
    value = intercountry_nb_mentions_df_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)

In [11]:
folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        'fillColor': color_function(feature),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.GeoJsonTooltip(fields=['name'],aliases=['Country: '], sticky=True, style="font-family: Arial; color: black;", \
                           opacity=0.8, direction='top'),
    name = 'Number of events',
    ).add_to(world_map)

color_scale.caption = 'Number of events'
world_map.add_child(color_scale)
folium.LayerControl().add_to(world_map)
world_map

In [12]:
world_map = folium.Map(tiles='Mapbox bright', zoom_start=5.5)

color_scale2 = LinearColormap(['yellow','red'], vmin = min(intercountry_AVG_tone_df_dict.values()), \
                                               vmax = max(intercountry_AVG_tone_df_dict.values()) \
                            )

def color_function(feature):
    value = intercountry_AVG_tone_df_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale2(value)
    
folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        'fillColor': color_function(feature),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.GeoJsonTooltip(fields=['name'],aliases=['Country: '], sticky=True, style="font-family: Arial; color: black;", \
                           opacity=0.8, direction='top'),
    name = 'Country Media Average',
    ).add_to(world_map)

color_scale2.caption = 'Average Tone'
world_map.add_child(color_scale2)
folium.LayerControl().add_to(world_map)
world_map    